<a href="https://colab.research.google.com/github/navyaboddapati/LLM--Detect-AI-Generated-Text/blob/main/LLM_Detect_AI_Generated_Text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle

In [2]:
import os
os.environ['KAGGLE_USERNAME'] = 'navyaboddapati'
os.environ['KAGGLE_KEY'] = '3d801ca7ef63201eb65ce89d7af7db3d'

In [3]:
!kaggle competitions download -c llm-detect-ai-generated-text

  0% 0.00/1.38M [00:00<?, ?B/s]
100% 1.38M/1.38M [00:00<00:00, 84.1MB/s]


In [4]:
!unzip llm-detect-ai-generated-text.zip

Archive:  llm-detect-ai-generated-text.zip
  inflating: sample_submission.csv   
  inflating: test_essays.csv         
  inflating: train_essays.csv        
  inflating: train_prompts.csv       


In [5]:
import pandas as pd

In [6]:
# Loading the data
train_data = pd.read_csv('train_essays.csv')

In [7]:
print('Head train data : ')
print(train_data.head())
print('I\nnfo: ')
print(train_data.info())
print('\nDescribe data: ')
print(train_data.describe())

Head train data : 
         id  prompt_id                                               text  \
0  0059830c          0  Cars. Cars have been around since they became ...   
1  005db917          0  Transportation is a large necessity in most co...   
2  008f63e3          0  "America's love affair with it's vehicles seem...   
3  00940276          0  How often do you ride in a car? Do you drive a...   
4  00c39458          0  Cars are a wonderful thing. They are perhaps o...   

   generated  
0          0  
1          0  
2          0  
3          0  
4          0  
I
nfo: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1378 entries, 0 to 1377
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         1378 non-null   object
 1   prompt_id  1378 non-null   int64 
 2   text       1378 non-null   object
 3   generated  1378 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 43.2+ KB
None

Describe data: 
       

In [8]:
# finding no. of records
num_records = len(train_data)
print(f'Number of Records: {num_records}')

Number of Records: 1378


In [9]:
# no. of null values in the data
null_counts = train_data.isnull().sum()
print(f'Number of Null Values:{null_counts}')

Number of Null Values:id           0
prompt_id    0
text         0
generated    0
dtype: int64


In [10]:
'''df1 = pd.read_csv('train_essays.csv')
df2 = pd.read_csv('train_prompts.csv')
df = pd.merge(df1,df2,)'''

"df1 = pd.read_csv('train_essays.csv')\ndf2 = pd.read_csv('train_prompts.csv')\ndf = pd.merge(df1,df2,)"

In [11]:
# checking for how many are human essays and AI generated essays
import matplotlib.pyplot as plt
import seaborn as sns

generated_Counts = train_data['generated'].value_counts()

plt.figure(figsize=(8,6))
sns.barplot(x=generated_counts.index, y=generated_counts.values, palette='viridis')
plt.title('Distribution of Human-Generated and AI-Generated Essays')
plt.xlabel('Generated (0: Human, 1:AI)')
plt.ylabel('Number of Essays')
plt.show()

NameError: ignored

<Figure size 800x600 with 0 Axes>